In [7]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

In [8]:
# Cargar el CSV
prices = pd.read_csv("prices/AAPL_prices.csv", index_col=0, parse_dates=True)
prices.head(3)

,Open,High,Low,Close,Volume
Date,,,,,
2005-01-03,1.156786,1.162679,1.117857,1.130179,691992000
2005-01-04,1.139107,1.169107,1.124464,1.141786,1096810400
2005-01-05,1.151071,1.165179,1.143750,1.151786,680433600


# ***1. Preprocesamiento de Datos***

Se aplica Min-Max scaling para cada stock price $x$. El valor normalizado $x$ se calcula:

$$x'= \frac{x-x_{min}}{x_{max}-x_{min}}$$

La normalización es esencial para la convergencia de la red neuronal (LSTM-GNN paper).

Predecir el valor normalizado conviene matematicamente. La red aprende mejor en el rango [0,1] y después puede revertirse al precio real.

El $x_{min}$ y el $x_{max}$ deben obtenerse de los datos del entrenamiento porque sino estariamos "observando el futuro", lo que se denomina como *data leakage*.

Por ejemplo, si entrenaramos con datos de 2005-2020, pero normalizaste usando max y min de 2005-2025, el modelo sabrá, indirectamente, que el precio nunca bajará de cierto valor en 2024.

En cada paso de entrenamiento, se recalcula el Min-Max, solo con los datos pasados hasta ese momento.

Se utiliza:
- **Expading window**: Cuanto pasado conoce el modelo en total
- **Window size**: Cuanto pasado mira en cada predicción

## **Expanding Window & Rolling Window**

En el paper LSTM+GNN hacen *expanding window*. El conjunto de entrenamiento crece día a día y se reentrena el modelo antes de predecir el siguiente día.

Para predecir el día **t**:
1. Entreno todos los datos hasta **t-1** (solo se observa el pasado).
2. Escalo con Min-Max calculado solo en ese train set (sin leakage).
3. Construyo ventanas (de tamaño W) dentro de ese train.
4. Armo el sample test del día **t** con su ventana [t-W, …, t-1] escalada usando el mismo Min-Max del train de ese paso.
5. Se entrena el LSTM y se predice el cierre normalizado en **t**.
6. Avanzo **t+1**, agregando el dato real de **t** al train, y se repite.

In [14]:
def build_xy_from_series(series: np.ndarray, window: int):
    X, y = [], []
    for t in range(window, len(series)):
        X.append(series[t-window:t].reshape(-1, 1))  # for empieza en W porque necesita los W valores previos.
        y.append(series[t])     # va desde t-W hasta t-1
    if not X:
        return (np.empty((0, window, 1), np.float32),
                np.empty((0,), np.float32))
    return np.array(X, np.float32), np.array(y, np.float32)

In [ ]:
def make_train_and_test_for_day(df: pd.DataFrame, day: pd.Timestamp, W: int):
    """
    Arma el dataset de TRAIN (2 años previos a day) y la muestra de TEST (la ventana que termina en day-1).
    Normaliza con Min–Max calculado SOLO en el train de este día (sin leakage).
    """
    eps = 1e-12
    train_start = day - pd.DateOffset(years=2)
    train_end   = day - pd.DateOffset(days=1)

    train_df = df.loc[train_start:train_end].copy()
    assert len(train_df) > W, "No hay suficientes datos en los 2 años previos."

    # Min–Max SOLO del train de este paso
    cmin = float(train_df["Close"].min())
    cmax = float(train_df["Close"].max())
    cscale = (cmax - cmin) if (cmax - cmin) > eps else eps

    # Normalizo train y generar ventanas (X_train, y_train)
    train_norm = ((train_df["Close"].values - cmin) / cscale).astype(np.float32)
    X_train, y_train = build_xy_from_series(train_norm, W)  # (N, W, 1), (N,)

    # Genero la window para el día de test, que va de [day-W : day-1]
    past_window_raw = df.loc[: day - pd.tseries.offsets.BDay(1), "Close"].tail(W).values

    assert len(past_window_raw) == W, "No hay W observaciones inmediatamente anteriores al día de test."
    X_test = (((past_window_raw - cmin) / cscale).astype(np.float32)).reshape(1, W, 1)

    y_raw  = df.loc[day, "Close"]
    y_test = np.array([(y_raw - cmin) / cscale], dtype=np.float32)  # (1,)
    return X_train, y_train, X_test, y_test, (cmin, cscale), (train_start, train_end)


In [ ]:
df = prices.copy()
W = 50

def next_trading_day(day):
    if day not in df.index:   # si el día de testeo no esta disponible busca el siguiente día habil
        idx = df.index.get_indexer([day], method="bfill")[0]
        day = df.index[idx]
    return day 

day = pd.Timestamp("2023-01-01")
day = next_trading_day(day)   # me fijo si ese dia es de trading, sino paso al siguiente

Xtr, ytr, Xte, yte, (cmin, cscale), (ts, te), day_aligned = make_train_and_test_for_day(df, day, W)
print("TRAIN:", ts.date(), "→", te.date(),
      "| TEST day:", day_aligned.date(),
      "| Xtr:", Xtr.shape, "ytr:", ytr.shape, "| Xte:", Xte.shape, "yte:", yte.shape)

TRAIN: 2021-01-03 → 2023-01-02 | TEST day: 2023-01-03 | Xtr: (453, 50, 1) ytr: (453,) | Xte: (1, 50, 1) yte: (1,)
